In [1]:
import os
import pandas as pd
import json

In [2]:
def read_ad(filename):
    try:
        with open(filename+'/meta.json') as file:
            data = json.load(file)
        with open(filename+'/activities.json') as file:
            activities_data = json.load(file)
    except:
        return None
    # Filter data to include only types 'i', 'c', or 'p'
    filtered_data = [item for item in data if item['type'] in ['i', 'c', 'p']]
    
    # Create a DataFrame with the type and the dict format of the element
    if len(filtered_data)==0:
        return None
    df = pd.DataFrame(filtered_data)
    df['element_dict'] = df['element'].apply(lambda x: json.dumps(x))
    df_result = df[['type', 'element_dict']]
        # Extract the package name and activity from the activities data
    df_activities = pd.DataFrame(activities_data, columns=['package_name', 'activity'])
    
    # Merge the activities DataFrame with the filtered data DataFrame based on index
    # Assuming each index in df_result corresponds directly to an index in df_activities
    df_result = df_result.join(df_activities)
    return df_result

In [3]:
dfs=[]
for app_name in os.listdir('groundtruth/'):
    if app_name[0]=='.':
        continue #skip notebook .checkpoints
    df_result=read_ad('groundtruth/'+app_name)
    dfs.append(df_result)
df=pd.concat(dfs,axis=0)

In [5]:
df

,type,element_dict,package_name,activity
0,p,"{""index"": ""0"", ""text"": """", ""resource-id"": """", ...",paint.by.number.pixel.art.coloring.drawing.puzzle,com.meevii.business.color.draw.ColorDrawActivity
1,i,"{""index"": ""1"", ""text"": """", ""resource-id"": ""pai...",paint.by.number.pixel.art.coloring.drawing.puzzle,com.meevii.business.color.draw.ColorDrawActivity
0,i,"{""index"": ""1"", ""text"": """", ""resource-id"": ""com...",com.dicionario_bibico_free.dicionario_bibico_free,com.zavarise.aplicativos.atividades.main.MainA...
1,c,"{""index"": ""1"", ""text"": """", ""resource-id"": """", ...",com.dicionario_bibico_free.dicionario_bibico_free,com.zavarise.aplicativos.atividades.main.MainA...
2,i,"{""index"": ""0"", ""text"": """", ""resource-id"": ""com...",com.dicionario_bibico_free.dicionario_bibico_free,com.zavarise.aplicativos.atividades.main.MainA...
0,i,"{""index"": ""0"", ""text"": """", ""resource-id"": ""com...",com.kitegames.baby.spans.photo.editor,com.kitegames.babysnaps.controllers.activities...
1,p,"{""index"": ""2"", ""text"": """", ""resource-id"": """", ...",com.kitegames.baby.spans.photo.editor,com.kitegames.babysnaps.controllers.activities...
2,c,"{""index"": ""1"", ""text"": """", ""resource-id"": ""com...",com.kitegames.baby.spans.photo.editor,com.kitegames.babysnaps.controllers.activities...
3,c,"{""index"": ""0"", ""text"": """", ""resource-id"": ""com...",com.kitegames.baby.spans.photo.editor,com.kitegames.babysnaps.controllers.activities...


In [11]:
mapping_dict = {'p': 'popup', 'i': 'adview', 'c': 'custom-made'}
df['type'] = df['type'].apply(lambda x: mapping_dict.get(x, x))
df.type.value_counts()

adview         4
custom-made    3
popup          2
Name: type, dtype: int64